# LLM Optimization Modelling Experiment

In [156]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel
from IPython.display import Markdown

## 1. Define the problem description

In [563]:
problem = '''A buyer needs to acquire 239,600,480 units of a product and is considering bids from five suppliers, labeled A through E, each of whom can only supply a portion of the total required amount.
Each vendor has proposed different pricing structures, incorporating both setup fees and variable unit costs that change based on the quantity ordered.

The buyer's objective is to allocate the order among these suppliers to minimize overall costs, accounting for both setup and unit costs.

Vendor A offers a set up cost of $3855.34 and a unit cost of $61.150 per thousand of units.
Vendor A can supply up to 33 million units.

Vendor B offers a set up cost of $125,804.84 if purchasing between 22,000,000-70,000,000 units from vendor B with a unit cost of $68.099 per thousand units.
If purchasing between 70,000,001-100,000,000 units from vendor B, the set up cost increases to $269304.84 and the unit cost sinks to $66.049 per thousand units.
If purchasing between 100,000,001-150,000,000 units from vendor B, the unit cost per thousand units further decreases to $64.099, but the set up cost increases to $464304.84.
If purchasing between 150,000,001 and 160,000,000 units from vendor B, the unit cost is $62.119 per thousand units and the set up cost equals $761304.84.

Vendor C offers set up costs of $13,456.00 and a unit cost of $62.019 per thousand units.
Vendor C can supply up to 165.6 million units. Vendor D offers set up costs of $6,583.98 and a unit cost of $72.488 for a set of thousand units.

Vendor D can supply up to 12 million units at a price of $72.488 per thousand units and with a set up cost of $6583.98.

Vendor E offers free set up if purchasing between 0 and 42 million units of vendor E with a unit price of $70.150 per thousand units.
If purchasing between 42,000,001 and 77 million units from vendor E, the unit cost starts at $68.150 per thousand units, but with every one million units purchased the price decreases at a rate of 0.05 percent. An additional set up cost of $84000 will be charged as well.

Note that zero units may be purchased from vendor B: otherwise no positive number of units less than 22,000,000 may be purchased.'''

## 2. Generate the mathematical model

In [581]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Let's think step by step. Please write a mathematical optimization model for this problem. If there are parameter values, make sure to include them in the mathematical formulation.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [582]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model

**Sets:**

*  $V = \{A, B, C, D, E\}$: Set of vendors
*  $T_B = \{1, 2, 3, 4\}$: Set of pricing tiers for vendor B
* $T_E = \{1, 2\}$: Set of pricing tiers for vendor E 

**Parameters:**

* $R = 239,\!600,\!480$: Total units required
* $S_v$: Setup cost for vendor $v \in V$
    * $S_A = \$3,\!855.34$
    * $S_B = \{\$125,\!804.84, \$269,\!304.84, \$464,\!304.84, \$761,\!304.84\}$ for each tier in $T_B$ respectively
    * $S_C = \$13,\!456.00$
    * $S_D = \$6,\!583.98$
    * $S_E = \{\$0, \$84,\!000\}$ for each tier in $T_E$ respectively
* $C_{v,t}$: Unit cost per thousand units for vendor $v \in V$ at pricing tier $t$ (if applicable)
    * $C_A = \$61.150$
    * $C_{B,t} = \{\$68.099, \$66.049, \$64.099, \$62.119\}$ for each tier in $T_B$ respectively 
    * $C_C = \$62.019$
    * $C_D = \$72.488$
    * $C_{E,1} = \$70.150$
* $L_{v,t}$: Lower bound on the number of units purchased from vendor $v \in V$ at pricing tier $t$ (if applicable)
    * $L_{B,1} = 22,\!000,\!000$
    * $L_{B,2} = 70,\!000,\!001$
    * $L_{B,3} = 100,\!000,\!001$
    * $L_{B,4} = 150,\!000,\!001$
    * $L_{E,1} = 0$
    * $L_{E,2} = 42,\!000,\!001$
* $U_{v,t}$: Upper bound on the number of units purchased from vendor $v \in V$ at pricing tier $t$ (if applicable)
    * $U_A = 33,\!000,\!000$
    * $U_{B,1} = 70,\!000,\!000$
    * $U_{B,2} = 100,\!000,\!000$
    * $U_{B,3} = 150,\!000,\!000$
    * $U_{B,4} = 160,\!000,\!000$
    * $U_C = 165,\!600,\!000$
    * $U_D = 12,\!000,\!000$
    * $U_{E,1} = 42,\!000,\!000$
    * $U_{E,2} = 77,\!000,\!000$

**Decision Variables:**

* $x_{v,t}$: Number of units purchased from vendor $v \in V$ at pricing tier $t$ (if applicable). 
* $y_{v,t}$: Binary variable equal to 1 if purchasing from vendor $v \in V$ at pricing tier $t$, and 0 otherwise.

**Objective Function:** 
Minimize the total cost:

```
Minimize Z = ∑_{v∈V} ∑_{t∈T_v} (S_v * y_{v,t} + (C_{v,t}/1000) * x_{v,t} ) 
```

**Constraints:**

1. **Demand Satisfaction:** The total number of units purchased must meet the required demand.
   ```
   ∑_{v∈V} ∑_{t∈T_v} x_{v,t} = R 
   ```

2. **Vendor Capacity:** The number of units purchased from each vendor must be within their capacity limits for each tier.
   ```
   L_{v,t} * y_{v,t}  ≤ x_{v,t} ≤  U_{v,t} * y_{v,t} ,  ∀ v∈V, ∀ t∈T_v
   ```

3. **Vendor B Tier Selection:** At most one tier can be selected for vendor B.
   ```
   ∑_{t∈T_B} y_{B,t} ≤ 1
   ```

4. **Vendor E Tier Selection:** At most one tier can be selected for vendor E.
    ```
    ∑_{t∈T_E} y_{E,t} ≤ 1 
    ```

5. **Binary Variable Restriction:** 
   ```
   y_{v,t} ∈ {0, 1}, ∀ v∈V, ∀ t∈T_v 
   ```

**Notes:**
* The unit costs are given per thousand units, so they are divided by 1000 in the objective function.
* The model uses binary variables ($y_{v,t}$) to activate or deactivate the setup costs and unit costs for each vendor and tier.
* The model assumes that the price decrease for vendor E in tier 2 is applied linearly over the entire range of units within that tier. 


## 3. Generate the pyomo code

In [583]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [584]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Define sets
V = ['A', 'B', 'C', 'D', 'E']  # Vendors
T_B = [1, 2, 3, 4]  # Pricing tiers for vendor B
T_E = [1, 2]  # Pricing tiers for vendor E

# Define parameters
R = 239600480  # Total units required

S = {
    'A': 3855.34,
    'B': {1: 125804.84, 2: 269304.84, 3: 464304.84, 4: 761304.84},
    'C': 13456.00,
    'D': 6583.98,
    'E': {1: 0, 2: 84000}
}  # Setup costs

C = {
    'A': 61.150,
    'B': {1: 68.099, 2: 66.049, 3: 64.099, 4: 62.119},
    'C': 62.019,
    'D': 72.488,
    'E': {1: 70.150}  # Note: C[E][2] is not needed, see note in problem description
}  # Unit costs per thousand units

L = {
    'B': {1: 22000000, 2: 70000001, 3: 100000001, 4: 150000001},
    'E': {1: 0, 2: 42000001}
}  # Lower bounds on units purchased

U = {
    'A': 33000000,
    'B': {1: 70000000, 2: 100000000, 3: 150000000, 4: 160000000},
    'C': 165600000,
    'D': 12000000,
    'E': {1: 42000000, 2: 77000000}
}  # Upper bounds on units purchased

# Create a concrete model
model = pyo.ConcreteModel()

# Define decision variables
model.x = pyo.Var(((v, t) for v in V for t in (T_B if v == 'B' else T_E if v == 'E' else [1])), within=pyo.NonNegativeReals)
model.y = pyo.Var(((v, t) for v in V for t in (T_B if v == 'B' else T_E if v == 'E' else [1])), within=pyo.Binary)

# Define objective function
model.obj = pyo.Objective(expr=sum(S[v][t] * model.y[v, t] + (C[v][t] / 1000) * model.x[v, t]
                         for v in V for t in (T_B if v == 'B' else T_E if v == 'E' else [1])), sense=pyo.minimize)

# Define constraints
model.demand = pyo.Constraint(expr=sum(model.x[v, t] for v in V for t in (T_B if v == 'B' else T_E if v == 'E' else [1])) == R)

def capacity_rule(model, v, t):
    return L[v][t] * model.y[v, t] <= model.x[v, t] <= U[v][t] * model.y[v, t]

model.vendor_capacity = pyo.Constraint(
    [(v, t) for v in ['B', 'E'] for t in (T_B if v == 'B' else T_E)], rule=capacity_rule)

model.vendor_capacity_A = pyo.Constraint(expr=model.x['A', 1] <= U['A'] * model.y['A', 1])
model.vendor_capacity_C = pyo.Constraint(expr=model.x['C', 1] <= U['C'] * model.y['C', 1])
model.vendor_capacity_D = pyo.Constraint(expr=model.x['D', 1] <= U['D'] * model.y['D', 1])

model.vendor_B_tier = pyo.Constraint(expr=sum(model.y['B', t] for t in T_B) <= 1)
model.vendor_E_tier = pyo.Constraint(expr=sum(model.y['E', t] for t in T_E) <= 1)

# Solve the model
solver = pyo.SolverFactory('cbc') # You can change the solver if needed
results = solver.solve(model)

# Print the results
print(results)

# Print the optimal solution
print("Optimal Solution:")
for v in V:
    for t in (T_B if v == 'B' else T_E if v == 'E' else [1]):
        if pyo.value(model.y[v, t]) > 0:
            print(f"  Vendor {v}, Tier {t}: {pyo.value(model.x[v, t])} units")
```

**Notes:**

* I have used the sample data you provided for the parameters.
* The code defines the objective function and all the constraints you described.
* The `capacity_rule` function is used to define the capacity constraints for vendors B and E for all their respective tiers. 
* Separate constraints are defined for vendors A, C, and D as they have only one tier.
* The model is solved using the CBC solver, but you can change this to any solver you prefer.
* The code prints the solver status and the optimal solution, showing the number of units purchased from each vendor and tier. 


## 4. Input problem data and try running the generated code

In [585]:
import pyomo.environ as pyo

# Define sets
V = ['A', 'B', 'C', 'D', 'E']  # Vendors
T_B = [1, 2, 3, 4]  # Pricing tiers for vendor B
T_E = [1, 2]  # Pricing tiers for vendor E

# Define parameters
R = 239600480  # Total units required

S = {
    'A': 3855.34,
    'B': {1: 125804.84, 2: 269304.84, 3: 464304.84, 4: 761304.84},
    'C': 13456.00,
    'D': 6583.98,
    'E': {1: 0, 2: 84000}
}  # Setup costs

C = {
    'A': 61.150,
    'B': {1: 68.099, 2: 66.049, 3: 64.099, 4: 62.119},
    'C': 62.019,
    'D': 72.488,
    'E': {1: 70.150}  # Note: C[E][2] is not needed, see note in problem description
}  # Unit costs per thousand units

L = {
    'B': {1: 22000000, 2: 70000001, 3: 100000001, 4: 150000001},
    'E': {1: 0, 2: 42000001}
}  # Lower bounds on units purchased

U = {
    'A': 33000000,
    'B': {1: 70000000, 2: 100000000, 3: 150000000, 4: 160000000},
    'C': 165600000,
    'D': 12000000,
    'E': {1: 42000000, 2: 77000000}
}  # Upper bounds on units purchased

# Create a concrete model
model = pyo.ConcreteModel()

# Define decision variables
model.x = pyo.Var(((v, t) for v in V for t in (T_B if v == 'B' else T_E if v == 'E' else [1])), within=pyo.NonNegativeReals)
model.y = pyo.Var(((v, t) for v in V for t in (T_B if v == 'B' else T_E if v == 'E' else [1])), within=pyo.Binary)

# Define objective function
model.obj = pyo.Objective(expr=sum(S[v][t] * model.y[v, t] + (C[v][t] / 1000) * model.x[v, t]
                         for v in V for t in (T_B if v == 'B' else T_E if v == 'E' else [1])), sense=pyo.minimize)

# Define constraints
model.demand = pyo.Constraint(expr=sum(model.x[v, t] for v in V for t in (T_B if v == 'B' else T_E if v == 'E' else [1])) == R)

def capacity_rule(model, v, t):
    return L[v][t] * model.y[v, t] <= model.x[v, t] <= U[v][t] * model.y[v, t]

model.vendor_capacity = pyo.Constraint(
    [(v, t) for v in ['B', 'E'] for t in (T_B if v == 'B' else T_E)], rule=capacity_rule)

model.vendor_capacity_A = pyo.Constraint(expr=model.x['A', 1] <= U['A'] * model.y['A', 1])
model.vendor_capacity_C = pyo.Constraint(expr=model.x['C', 1] <= U['C'] * model.y['C', 1])
model.vendor_capacity_D = pyo.Constraint(expr=model.x['D', 1] <= U['D'] * model.y['D', 1])

model.vendor_B_tier = pyo.Constraint(expr=sum(model.y['B', t] for t in T_B) <= 1)
model.vendor_E_tier = pyo.Constraint(expr=sum(model.y['E', t] for t in T_E) <= 1)

# Solve the model
solver = pyo.SolverFactory('cbc') # You can change the solver if needed
results = solver.solve(model)

# Print the results
print(results)

# Print the optimal solution
print("Optimal Solution:")
for v in V:
    for t in (T_B if v == 'B' else T_E if v == 'E' else [1]):
        if pyo.value(model.y[v, t]) > 0:
            print(f"  Vendor {v}, Tier {t}: {pyo.value(model.x[v, t])} units")

TypeError: 'float' object is not subscriptable

## 5. Correct the code to verify model viability (optional)

In [594]:
import pyomo.environ as pyo

# Define sets
V = ['A', 'B', 'C', 'D', 'E']  # Vendors
T_B = [1, 2, 3, 4]  # Pricing tiers for vendor B
T_E = [1, 2]  # Pricing tiers for vendor E

# Define parameters
R = 239600480  # Total units required

S = {
    'A': {1:3855.34, 2:99999999},
    'B': {1: 125804.84, 2: 269304.84, 3: 464304.84, 4: 761304.84},
    'C': {1:13456.00, 2:99999999},
    'D': {1:6583.98, 2:99999999},
    'E': {1: 0, 2: 84000}
}  # Setup costs

C = {
    'A': {1:61.150, 2:99999999},
    'B': {1: 68.099, 2: 66.049, 3: 64.099, 4: 62.119},
    'C': {1:62.019, 2:99999999},
    'D': {1:72.488, 2:99999999},
    'E': {1: 70.150, 2:99999999}  # Note: C[E][2] is not needed, see note in problem description
}  # Unit costs per thousand units

L = {
    'B': {1: 22000000, 2: 70000001, 3: 100000001, 4: 150000001},
    'E': {1: 0, 2: 42000001}
}  # Lower bounds on units purchased

U = {
    'A': {1:33000000},
    'B': {1: 70000000, 2: 100000000, 3: 150000000, 4: 160000000},
    'C': {1:165600000},
    'D': {1:12000000},
    'E': {1: 42000000, 2: 77000000}
}  # Upper bounds on units purchased

# Create a concrete model
model = pyo.ConcreteModel()

# Define decision variables
model.x = pyo.Var(((v, t) for v in V for t in (T_B if v == 'B' else T_E if v == 'E' else [1])), within=pyo.NonNegativeReals)
model.y = pyo.Var(((v, t) for v in V for t in (T_B if v == 'B' else T_E if v == 'E' else [1])), within=pyo.Binary)

# Define objective function
model.obj = pyo.Objective(expr=sum(S[v][t] * model.y[v, t] + (C[v][t] / 1000) * model.x[v, t]
                         for v in V for t in (T_B if v == 'B' else T_E if v == 'E' else [1])), sense=pyo.minimize)

# Define constraints
model.demand = pyo.Constraint(expr=sum(model.x[v, t] for v in V for t in (T_B if v == 'B' else T_E if v == 'E' else [1])) == R)

def lower_capacity_rule(model, v, t):
    return L[v][t] * model.y[v, t] <= model.x[v, t]

def upper_capacity_rule(model, v, t):
    return model.x[v, t] <= U[v][t] * model.y[v, t]

model.lower_vendor_capacity = pyo.Constraint(
    [(v, t) for v in ['B', 'E'] for t in (T_B if v == 'B' else T_E)], rule=lower_capacity_rule)

model.upper_vendor_capacity = pyo.Constraint(
    [(v, t) for v in ['B', 'E'] for t in (T_B if v == 'B' else T_E)], rule=upper_capacity_rule)


model.vendor_capacity_A = pyo.Constraint(expr=model.x['A', 1] <= U['A'][1] * model.y['A', 1])
model.vendor_capacity_C = pyo.Constraint(expr=model.x['C', 1] <= U['C'][1] * model.y['C', 1])
model.vendor_capacity_D = pyo.Constraint(expr=model.x['D', 1] <= U['D'][1] * model.y['D', 1])

model.vendor_B_tier = pyo.Constraint(expr=sum(model.y['B', t] for t in T_B) <= 1)
model.vendor_E_tier = pyo.Constraint(expr=sum(model.y['E', t] for t in T_E) <= 1)

# Solve the model
solver = pyo.SolverFactory('glpk') # You can change the solver if needed
results = solver.solve(model)

# Print the results
print(results)

# Print the optimal solution
print("Optimal Solution:")
for v in V:
    for t in (T_B if v == 'B' else T_E if v == 'E' else [1]):
        if pyo.value(model.y[v, t]) > 0:
            print(f"  Vendor {v}, Tier {t}: {pyo.value(model.x[v, t])} units")


Problem: 
- Name: unknown
  Lower bound: 15181791.412
  Upper bound: 15181791.412
  Number of objectives: 1
  Number of constraints: 18
  Number of variables: 18
  Number of nonzeros: 44
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 11
      Number of created subproblems: 11
  Error rc: 0
  Time: 0.04854440689086914
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Optimal Solution:
  Vendor A, Tier 1: 33000000.0 units
  Vendor C, Tier 1: 165600000.0 units
  Vendor E, Tier 1: 41000480.0 units


In [595]:
model.obj()

15181791.412

## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [596]:
print(response.text)

## Mathematical Optimization Model

**Sets:**

*  $V = \{A, B, C, D, E\}$: Set of vendors
*  $T_B = \{1, 2, 3, 4\}$: Set of pricing tiers for vendor B
* $T_E = \{1, 2\}$: Set of pricing tiers for vendor E 

**Parameters:**

* $R = 239,\!600,\!480$: Total units required
* $S_v$: Setup cost for vendor $v \in V$
    * $S_A = \$3,\!855.34$
    * $S_B = \{\$125,\!804.84, \$269,\!304.84, \$464,\!304.84, \$761,\!304.84\}$ for each tier in $T_B$ respectively
    * $S_C = \$13,\!456.00$
    * $S_D = \$6,\!583.98$
    * $S_E = \{\$0, \$84,\!000\}$ for each tier in $T_E$ respectively
* $C_{v,t}$: Unit cost per thousand units for vendor $v \in V$ at pricing tier $t$ (if applicable)
    * $C_A = \$61.150$
    * $C_{B,t} = \{\$68.099, \$66.049, \$64.099, \$62.119\}$ for each tier in $T_B$ respectively 
    * $C_C = \$62.019$
    * $C_D = \$72.488$
    * $C_{E,1} = \$70.150$
* $L_{v,t}$: Lower bound on the number of units purchased from vendor $v \in V$ at pricing tier $t$ (if applicable)
    * $L_{B

In [597]:
print(response2.text)

```python
import pyomo.environ as pyo

# Define sets
V = ['A', 'B', 'C', 'D', 'E']  # Vendors
T_B = [1, 2, 3, 4]  # Pricing tiers for vendor B
T_E = [1, 2]  # Pricing tiers for vendor E

# Define parameters
R = 239600480  # Total units required

S = {
    'A': 3855.34,
    'B': {1: 125804.84, 2: 269304.84, 3: 464304.84, 4: 761304.84},
    'C': 13456.00,
    'D': 6583.98,
    'E': {1: 0, 2: 84000}
}  # Setup costs

C = {
    'A': 61.150,
    'B': {1: 68.099, 2: 66.049, 3: 64.099, 4: 62.119},
    'C': 62.019,
    'D': 72.488,
    'E': {1: 70.150}  # Note: C[E][2] is not needed, see note in problem description
}  # Unit costs per thousand units

L = {
    'B': {1: 22000000, 2: 70000001, 3: 100000001, 4: 150000001},
    'E': {1: 0, 2: 42000001}
}  # Lower bounds on units purchased

U = {
    'A': 33000000,
    'B': {1: 70000000, 2: 100000000, 3: 150000000, 4: 160000000},
    'C': 165600000,
    'D': 12000000,
    'E': {1: 42000000, 2: 77000000}
}  # Upper bounds on units purchased

# Creat